In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
import shutil, os

In [48]:
# Load calib results
proj_mtx =  np.load('../data/proj_mtx.npy', )
proj_dist = np.load('../data/proj_dist.npy', )
cam_mtx =   np.load('../data/mtx.npy', )
cam_dist =  np.load('../data/dist.npy')
proj_R =    np.load('../data/R.npy')
proj_T =    np.load('../data/T.npy')
R1 =        np.load('../data/R1.npy')
R2 =        np.load('../data/R2.npy')
P1 =        np.load('../data/P1.npy')
P2 =        np.load('../data/P2.npy')

scale_x = 1920 / 800
scale_y = 1080 / 600
proj_mtx[0, :] = proj_mtx[0, :] * scale_x
proj_mtx[1, :] = proj_mtx[1, :] * scale_y
print(proj_mtx)


base_path = '../data/recordings/record_19/'
h_pixels = np.load(base_path+'h_pixels.npy')
v_pixels = np.load(base_path+'v_pixels.npy')

[[8.45917079e+03 0.00000000e+00 2.16765609e+03]
 [0.00000000e+00 6.16108341e+03 1.00128295e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


In [137]:
arrwidth = 800
arrheight = 600
result_img = np.zeros((h_pixels.shape[0], h_pixels.shape[1], 3), dtype=int)

cam_pts = []
proj_pts = []

for i in range(750,1100):
    for j in range(400,900):
        h_value = h_pixels[j, i]
        v_value = v_pixels[j, i]
        if h_value == -1 or v_value == -1:
            result_img[j, i] = [0, 0, 0]
        else:
            cam_pts.append([j, i])
            h_value = min(arrwidth-1, h_value)
            v_value = min(arrheight-1, v_value)
            proj_pts.append([ h_value, v_value])
            # print((j, i))
            # print((v_value, h_value))

cam_pts = np.array(cam_pts, dtype=np.float32)
proj_pts = np.array(proj_pts, dtype=np.float32)

In [145]:
test = np.expand_dims(cam_pts, axis=1)
print(test[0])

[[400. 750.]]


In [154]:
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(cam_mtx, cam_dist, (1920,1080), 1, (1920,1080))
newprojmtx, roi = cv2.getOptimalNewCameraMatrix(proj_mtx, proj_dist, (800,600), 1, (800,600))
cam_pts_homo = cv2.undistortPoints(np.expand_dims(cam_pts, axis=1), cam_mtx, cam_dist, None, newcameramtx)
proj_pts_homo = cv2.undistortPoints(np.expand_dims(proj_pts, axis=1), proj_mtx, proj_dist, None, newprojmtx)

In [155]:
print(proj_pts_homo[0])

[[320. 372.]]


In [156]:
p4d = cv2.triangulatePoints(P1, P2, cam_pts_homo, proj_pts_homo)

In [158]:
p3d_euc = p4d / p4d[3, :]
p3d_euc = p3d_euc[:3, :]
#p3d_euc = p3d_euc / 1000
print(p3d_euc[:, 0])

[ -7.325365  -10.5152445 -12.044128 ]


In [159]:
import open3d as o3d
import open3d.core as o3c
pcd = o3d.t.geometry.PointCloud(o3c.Tensor(p3d_euc.T, o3c.float32))

o3d.visualization.draw_geometries([pcd.to_legacy()])